
### Requisitos para os arquivos da pasta partidas

Objetivo: realizar processo de ETL e disponibilizar os dados em camada adequada.

1. Ler o arquivos csv da pasta no Datalake [ok]
2. Renomear colunas para português usando snake case [ok]
3. Definir a tipagem de colunas correta [ok]
4. Coluna com data de cosumo do arquivo [ok]
5. Salvar dados em formato parquet na camada adequada [ok]

In [1]:
%run ../../Configuração-e-Utilitários/Variaveis.ipynb

In [2]:
%run ../../Configuração-e-Utilitários/Funções.ipynb

### Lendo os arquivos

In [5]:
partidas_df = (spark.read
               .option("header", "true")
               .option("inferSchema", "true")
               .option("encoding", "UTF-8")
               .csv(f"{caminho_bronze}/partidas_*.csv")  # O asterisco representa qualquer coisa após o underline ("_")
).drop('_c0')

# Imprimindo o df
partidas_df.show()

+------+-------+-------------------+--------+--------------+---------+-------------+------------------+-------------+-------------+----------+----------+-------------------+-------------------+-------------------+-------------------+
|    id|referee|               date|venue.id|   status.long|league.id|league.season|      league.round|teams.home.id|teams.away.id|goals.home|goals.away|score.halftime.home|score.halftime.away|score.fulltime.home|score.fulltime.away|
+------+-------+-------------------+--------+--------------+---------+-------------+------------------+-------------+-------------+----------+----------+-------------------+-------------------+-------------------+-------------------+
|191547|   NULL|2011-05-21 18:30:00|    NULL|Match Finished|     71.0|       2011.0|Regular Season - 1|        127.0|        145.0|       4.0|       0.0|                1.0|                0.0|                4.0|               0.0�|
|191548|   NULL|2011-05-21 18:30:00|    NULL|Match Finished|    

In [6]:
partidas_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- referee: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- venue.id: double (nullable = true)
 |-- status.long: string (nullable = true)
 |-- league.id: double (nullable = true)
 |-- league.season: double (nullable = true)
 |-- league.round: string (nullable = true)
 |-- teams.home.id: double (nullable = true)
 |-- teams.away.id: double (nullable = true)
 |-- goals.home: double (nullable = true)
 |-- goals.away: double (nullable = true)
 |-- score.halftime.home: double (nullable = true)
 |-- score.halftime.away: double (nullable = true)
 |-- score.fulltime.home: double (nullable = true)
 |-- score.fulltime.away: string (nullable = true)



### Renomeando as colunas

In [7]:
partidas_df = partidas_df \
    .withColumnRenamed("id", "id_partida") \
    .withColumnRenamed("referee", "nome_juiz") \
    .withColumnRenamed("date", "data_partida") \
    .withColumnRenamed("venue.id", "id_estadio") \
    .withColumnRenamed("status.long", "status_da_partida") \
    .withColumnRenamed("league.id", "id_torneio") \
    .withColumnRenamed("league.season", "ano_da_temporada") \
    .withColumnRenamed("league.round", "nr_partida") \
    .withColumnRenamed("teams.home.id", "id_time_casa") \
    .withColumnRenamed("teams.away.id", "id_time_visitante") \
    .withColumnRenamed("goals.home", "gols_casa") \
    .withColumnRenamed("goals.away", "gols_visitante") \
    .withColumnRenamed("score.halftime.home", "gols_meio_tempo_casa") \
    .withColumnRenamed("score.halftime.away", "gols_meio_tempo_visitante") \
    .withColumnRenamed("score.fulltime.home", "gols_partida_casa") \
    .withColumnRenamed("score.fulltime.away", "gols_partida_visitante")

partidas_df.show()

+----------+---------+-------------------+----------+-----------------+----------+----------------+------------------+------------+-----------------+---------+--------------+--------------------+-------------------------+-----------------+----------------------+
|id_partida|nome_juiz|       data_partida|id_estadio|status_da_partida|id_torneio|ano_da_temporada|        nr_partida|id_time_casa|id_time_visitante|gols_casa|gols_visitante|gols_meio_tempo_casa|gols_meio_tempo_visitante|gols_partida_casa|gols_partida_visitante|
+----------+---------+-------------------+----------+-----------------+----------+----------------+------------------+------------+-----------------+---------+--------------+--------------------+-------------------------+-----------------+----------------------+
|    191547|     NULL|2011-05-21 18:30:00|      NULL|   Match Finished|      71.0|          2011.0|Regular Season - 1|       127.0|            145.0|      4.0|           0.0|                 1.0|                

### Correção do schema

In [8]:
partidas_df.printSchema()

root
 |-- id_partida: string (nullable = true)
 |-- nome_juiz: string (nullable = true)
 |-- data_partida: timestamp (nullable = true)
 |-- id_estadio: double (nullable = true)
 |-- status_da_partida: string (nullable = true)
 |-- id_torneio: double (nullable = true)
 |-- ano_da_temporada: double (nullable = true)
 |-- nr_partida: string (nullable = true)
 |-- id_time_casa: double (nullable = true)
 |-- id_time_visitante: double (nullable = true)
 |-- gols_casa: double (nullable = true)
 |-- gols_visitante: double (nullable = true)
 |-- gols_meio_tempo_casa: double (nullable = true)
 |-- gols_meio_tempo_visitante: double (nullable = true)
 |-- gols_partida_casa: double (nullable = true)
 |-- gols_partida_visitante: string (nullable = true)



In [9]:
from pyspark.sql.functions import substring

# Corrigindo valores da coluna gols_visitante
partidas_df = partidas_df.withColumn('gols_partida_visitante', substring(partidas_df.gols_visitante, 0, 1))
partidas_df.show()

+----------+---------+-------------------+----------+-----------------+----------+----------------+------------------+------------+-----------------+---------+--------------+--------------------+-------------------------+-----------------+----------------------+
|id_partida|nome_juiz|       data_partida|id_estadio|status_da_partida|id_torneio|ano_da_temporada|        nr_partida|id_time_casa|id_time_visitante|gols_casa|gols_visitante|gols_meio_tempo_casa|gols_meio_tempo_visitante|gols_partida_casa|gols_partida_visitante|
+----------+---------+-------------------+----------+-----------------+----------+----------------+------------------+------------+-----------------+---------+--------------+--------------------+-------------------------+-----------------+----------------------+
|    191547|     NULL|2011-05-21 18:30:00|      NULL|   Match Finished|      71.0|          2011.0|Regular Season - 1|       127.0|            145.0|      4.0|           0.0|                 1.0|                

In [10]:
# Alterando os tipos da colunas
partidas_df = partidas_df \
    .withColumn("data_partida",                 partidas_df.data_partida.cast("date")) \
    .withColumn("id_estadio",                   partidas_df.id_estadio.cast("integer")) \
    .withColumn("id_torneio",                   partidas_df.id_torneio.cast("integer")) \
    .withColumn("ano_da_temporada",             partidas_df.ano_da_temporada.cast("integer")) \
    .withColumn("id_time_casa",                 partidas_df.id_time_casa.cast("integer")) \
    .withColumn("id_time_visitante",            partidas_df.id_time_visitante.cast("integer")) \
    .withColumn("gols_meio_tempo_casa",         partidas_df.gols_meio_tempo_casa.cast("integer")) \
    .withColumn("gols_meio_tempo_visitante",    partidas_df.gols_meio_tempo_visitante.cast("integer")) \
    .withColumn("gols_partida_casa",            partidas_df.gols_partida_casa.cast("integer")) \
    .withColumn("gols_partida_visitante",       partidas_df.gols_partida_visitante.cast("integer")) \
    .withColumn("gols_casa",                    partidas_df.gols_casa.cast("integer")) \
    .withColumn("gols_visitante",               partidas_df.gols_visitante.cast("integer"))

partidas_df.show()

+----------+---------+------------+----------+-----------------+----------+----------------+------------------+------------+-----------------+---------+--------------+--------------------+-------------------------+-----------------+----------------------+
|id_partida|nome_juiz|data_partida|id_estadio|status_da_partida|id_torneio|ano_da_temporada|        nr_partida|id_time_casa|id_time_visitante|gols_casa|gols_visitante|gols_meio_tempo_casa|gols_meio_tempo_visitante|gols_partida_casa|gols_partida_visitante|
+----------+---------+------------+----------+-----------------+----------+----------------+------------------+------------+-----------------+---------+--------------+--------------------+-------------------------+-----------------+----------------------+
|    191547|     NULL|  2011-05-21|      NULL|   Match Finished|        71|            2011|Regular Season - 1|         127|              145|        4|             0|                   1|                        0|                4|

In [11]:
partidas_df.printSchema()

root
 |-- id_partida: string (nullable = true)
 |-- nome_juiz: string (nullable = true)
 |-- data_partida: date (nullable = true)
 |-- id_estadio: integer (nullable = true)
 |-- status_da_partida: string (nullable = true)
 |-- id_torneio: integer (nullable = true)
 |-- ano_da_temporada: integer (nullable = true)
 |-- nr_partida: string (nullable = true)
 |-- id_time_casa: integer (nullable = true)
 |-- id_time_visitante: integer (nullable = true)
 |-- gols_casa: integer (nullable = true)
 |-- gols_visitante: integer (nullable = true)
 |-- gols_meio_tempo_casa: integer (nullable = true)
 |-- gols_meio_tempo_visitante: integer (nullable = true)
 |-- gols_partida_casa: integer (nullable = true)
 |-- gols_partida_visitante: integer (nullable = true)



### Criando coluna com data de consumo

In [12]:
partidas_df = adicionar_data_de_ingestao(
    df = partidas_df,
    nome_coluna = 'data_de_ingestao',
    add_tempo = -3,
    unidade_de_medida = 'horas'
)

partidas_df.show()

+----------+---------+------------+----------+-----------------+----------+----------------+------------------+------------+-----------------+---------+--------------+--------------------+-------------------------+-----------------+----------------------+----------------+
|id_partida|nome_juiz|data_partida|id_estadio|status_da_partida|id_torneio|ano_da_temporada|        nr_partida|id_time_casa|id_time_visitante|gols_casa|gols_visitante|gols_meio_tempo_casa|gols_meio_tempo_visitante|gols_partida_casa|gols_partida_visitante|data_de_ingestao|
+----------+---------+------------+----------+-----------------+----------+----------------+------------------+------------+-----------------+---------+--------------+--------------------+-------------------------+-----------------+----------------------+----------------+
|    191547|     NULL|  2011-05-21|      NULL|   Match Finished|        71|            2011|Regular Season - 1|         127|              145|        4|             0|              

### Salvando em formato parquet

In [14]:
partidas_df.write.mode("Overwrite").parquet(f"{caminho_silver}/partidas")

In [15]:
dbutils.notebook.exit('Os arquivos da pasta "partidas" foram processados e carregados na camada silver.')